In [38]:
#imports, uploads, inspection of data
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", None, "display.max_columns", None)
np.random.seed(5)
file=r'/Users/cecylia/Desktop/Notes/winedata.csv'
df=pd.read_csv(file)
#this is a large dataset-lets take a smaller sample and drop anything with a null country, price, province
df=df.dropna(subset=['country', 'price', 'province', 'variety'])
#drop duplicates
df.drop_duplicates(subset=['title'], inplace=True)
#this is still a rather large dataset so lets take a  random sample of 40000
df1 = df.sample(n=40000)
df1.reset_index(inplace=True)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
#lets start building a simple recommender - skip from here
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df1['description'])

In [39]:
cosine_sim_matrix = linear_kernel(tfidf_matrix)

In [40]:
from rapidfuzz import process  # Faster fuzzy matching
import time
import ipywidgets as widgets

In [41]:
indices = pd.Series(df1.index, index=df1['title']).drop_duplicates()

In [88]:
pd.set_option('display.max_colwidth', None)  # Ensure full-length text

In [117]:
# Function that takes in wine title as input and outputs most similar titles - old one
def recommend(title, cosine_sim_matrix=cosine_sim_matrix):
    # Get the index of the wine that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all wine with that wine
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))

    # Sort the wines based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar wines
    sim_scores = sim_scores[1:11]

    # Get the wines indices
    wine_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar wines
    return df1['title'].iloc[wine_indices]

In [118]:
# Search function using fuzzy matching
def search_item(user_input, choices, threshold=80):
    """Find the closest match to user input using fuzzy matching."""
    best_match = process.extractOne(user_input, choices)
    return best_match[0] if best_match and best_match[1] >= threshold else None

In [119]:
# Interactive dropdown with fuzzy search
dropdown = widgets.Combobox(
    placeholder="Type to search...",
    options=df["title"].tolist(),  # Pre-fill with known items
    ensure_option=False,  # Allow users to type custom queries
)

In [120]:
# Output area
output = widgets.Output()

In [122]:
from IPython.core.display import display, HTML
import ipywidgets as widgets

output = widgets.Output()

def on_value_change(change):
    with output:
        output.clear_output()
        user_input = change['new']
        matched_item = search_item(user_input, df1["title"].tolist()) if user_input else None
        
        if matched_item:
            print(f"Showing results for: {matched_item}")

            recommendations = recommend(matched_item)  # Assuming this returns wine titles
            
            # Retrieve price and country for each recommended wine
            recommendation_details = df1[df1["title"].isin(recommendations)][["title", "price", "country", "points", "province", "variety"]]

            # Generate HTML with tooltips
            wine_html = "<ul>"
            for _, row in recommendation_details.iterrows():
                tooltip_text = f"Price: ${row['price']}, Country: {row['country']}, Points: {row['points']}, Province: {row['province']}, Variety: {row['variety']}"
                wine_html += f'<li><span title="{tooltip_text}">{row["title"]}</span></li>'
            wine_html += "</ul>"

            display(HTML(wine_html))  # Display recommendations with tooltips
        else:
            print("No close match found. Try another search.")

/var/folders/11/7dw54ln10md9k5cbk_wjyfy80000gn/T/ipykernel_21148/1671683592.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [123]:
# Attach event listener
dropdown.observe(on_value_change, names='value')

# Display widgets
display(dropdown, output)

Combobox(value='', options=('Quinta dos Avidagos 2011 Avidagos Red (Douro)', 'Rainstorm 2013 Pinot Gris (Willa…

Output()